In [3]:
import tensorflow as tf
import t3f

In [4]:
# x imitates a batch of inputs where each input 
# corresponds to 8 4-dimensionsal features
x = tf.placeholder(tf.float32, (None, 8, 4))

In [5]:
# split x into 8 4-dimensional vectors and reshape them
# in 3d tensors of size 1 x 4 x 1 so we can construct TT
crs_ = tf.unstack(x, axis=1)

In [6]:
tt_cores = []
for core in crs_:
    tt_cores.append(tf.reshape(core, (-1, 1, 4, 1)))

In [7]:
# now we can construct the feature tensor
# we use the TensorTrainBatch class (for each input we have a separate TT)

In [8]:
f_tensor = t3f.TensorTrainBatch(tt_cores)

In [9]:
print(f_tensor)

A (?) element batch of Tensor Trains of shape (4, 4, 4, 4, 4, 4, 4, 4), TT-ranks: (1, 1, 1, 1, 1, 1, 1, 1, 1)


In [10]:
# to perform regression with say 5 outputs we will create
# a TT variable of type TensorTrainBatch with batch size 5
# and then construct the matrix of inner products
# (total contractions of each TT in f_tensor and of each TT in the weights batch)

In [11]:
w_ = t3f.random_tensor_batch(shape=8 * [4,], tt_rank=2, batch_size=5)

w = t3f.get_variable('w', initializer=w_)

In [12]:
logits = t3f.pairwise_flat_inner(f_tensor, w)

In [13]:
# this is a tensor of size None x 5
# need to make a tiny fix in t3f.pairwise_flat_inner
# so the shape is correctly printed: 
# tf.squeeze(res) -> tf.squeeze(res, axis=[2, 3])
# in the return statement